In [ ]:
!pip install tensorflow_recommenders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 1.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
warkop_df = pd.read_csv('https://storage.googleapis.com/inwarkop-bucket/all%20warkop2.csv')

#encode categorical features
# label_encoder_region = LabelEncoder()
# warkop_df['region_encoded'] = label_encoder_region.fit_transform(warkop_df['region'])

facilities_set = set()
for facilities in warkop_df['facilities']:
    facilities_set.update(facilities.split(','))

for facility in facilities_set:
    warkop_df[facility] = warkop_df['facilities'].apply(lambda x: 1 if facility in x else 0)

# Normalize ratings
warkop_df['rating_normalized'] = warkop_df['rating'] / warkop_df['rating'].max()

In [ ]:
warkop_df.head()

,warkop_id,name,address,rating,rating_count,region,facilities,24hours,parking,wifi,ac,toilet,rating_normalized
0,1,WARKOP 2 SAUDARA,"Kota Jakarta Pusat, Daerah Khusus Ibukota Jakarta",4.5,2,Pusat,"24hours,wifi,toilet,parking",1,1,1,0,1,0.9
1,2,Warkop Selamat Sejahtera,"Jl. Kebon Kacang 30 No.15, RT.5/RW.4",4.5,2,Pusat,"wifi,toilet",0,0,1,0,1,0.9
2,3,Warkop Been Aouf,"Jl. Kb. Kacang XL No.24, RT.10/RW.4",4.5,68,Pusat,"toilet,parking",0,1,0,0,1,0.9
3,4,Warkop otoy,Jl. Kebon Kacang 30 No.7,5.0,3,Pusat,"toilet,parking",0,1,0,0,1,1.0
4,5,Warkop Barokah masjid jami'e darusalam,"Masjid Darussalam, Jl. Kotabumi Ujung No.Depan...",4.5,2,Pusat,"24hours,parking",1,1,0,0,0,0.9


In [ ]:
warkop_df["region"].unique()

array(['Pusat', 'Barat', 'Timur', 'Selatan', 'Utara'], dtype=object)

In [ ]:
class WarkopModel(tfrs.Model):

    def __init__(self):
        super().__init__()
        embedding_dim = 32

        # # Item model for facilities embeddings
        self.facilities_embedding = tf.keras.Sequential([
            tf.keras.layers.DenseFeatures([
                tf.feature_column.numeric_column(facility) for facility in facilities_set
            ]),
            tf.keras.layers.Dense(embedding_dim)
        ])

        # Rating prediction model
        self.rating_normalized = tf.keras.Sequential([
            tf.keras.layers.Dense(1)
        ])

        # Task setup
        self.task = tfrs.tasks.Ranking(
            loss = tf.keras.losses.BinaryCrossentropy(),
            metrics = [tf.keras.metrics.BinaryCrossentropy()]
        )

    def call(self, features):
        facilities_embeddings = self.facilities_embedding(features)
        return self.rating_normalized(facilities_embeddings)

    def compute_loss(self, features, training=False):
        labels = features.pop("rating_normalized")
        predictions = self(features)
        return self.task(labels=labels, predictions=predictions)

In [ ]:
# Prepare the data

# Split the data into train and test sets
train, test = train_test_split(warkop_df, test_size=0.2, random_state=42)

def preprocess(df):
    features = {name: np.array(value) for name, value in df.items()}
    return features

train = preprocess(train)
test = preprocess(test)

# Define the data pipeline
train = tf.data.Dataset.from_tensor_slices((train)).batch(32)
test = tf.data.Dataset.from_tensor_slices((test)).batch(32)

# Initialize and compile the model

# Compile the model
model = WarkopModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=1e-2))

# Train the model with Early Stopping
history = model.fit(train, epochs=100)

# Evaluate the model
model.evaluate(test)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


Epoch 1/100
7/7 [==============================] - 3s 14ms/step - binary_crossentropy: 14.4678 - loss: 14.4910 - regularization_loss: 0.0000e+00 - total_loss: 14.4910
Epoch 2/100
7/7 [==============================] - 0s 14ms/step - binary_crossentropy: 14.4678 - loss: 14.4910 - regularization_loss: 0.0000e+00 - total_loss: 14.4910
Epoch 3/100
7/7 [==============================] - 0s 17ms/step - binary_crossentropy: 14.4678 - loss: 14.4910 - regularization_loss: 0.0000e+00 - total_loss: 14.4910
Epoch 4/100
7/7 [==============================] - 0s 13ms/step - binary_crossentropy: 14.4678 - loss: 14.4910 - regularization_loss: 0.0000e+00 - total_loss: 14.4910
Epoch 5/100
7/7 [==============================] - 0s 18ms/step - binary_crossentropy: 14.4678 - loss: 14.4910 - regularization_loss: 0.0000e+00 - total_loss: 14.4910
Epoch 6/100
7/7 [==============================] - 0s 9ms/step - binary_crossentropy: 14.4678 - loss: 14.4910 - regularization_loss: 0.0000e+00 - total_loss: 14.4910

[14.40152359008789, 14.20719051361084, 0, 14.20719051361084]

In [ ]:
import matplotlib.pyplot as plt
# Extract training and validation metrics
train_loss = history.history['loss']
val_loss = history.history['val_loss']
# train_accuracy = history.history['binary_accuracy']
# val_accuracy = history.history['val_binary_accuracy']

# Plot Loss
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plot Accuracy
# plt.subplot(1, 2, 2)
# plt.plot(train_accuracy, label='Training Accuracy')
# plt.plot(val_accuracy, label='Validation Accuracy')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.title('Training and Validation Accuracy')
# plt.legend()

plt.tight_layout()
plt.show()


KeyError: 'val_loss'

In [ ]:
def recommend_warkop(input_features, top_k=50):
    # Create a copy of the DataFrame to avoid modifying the original
    warkop_df_copy = warkop_df.copy()

    # Convert input features to tensor
    input_features_tensor = {
        facility: tf.convert_to_tensor([input_features.get(facility, 0)], dtype=tf.float32)
        for facility in facilities_set
    }

    # Get predictions for all warkops
    warkop_inputs = {
        facility: tf.convert_to_tensor(warkop_df_copy[facility].values.reshape(-1, 1), dtype=tf.float32)
        for facility in facilities_set
    }

    predictions = model(warkop_inputs)

    # Add predictions to the copy DataFrame
    warkop_df_copy['prediction'] = predictions.numpy().flatten()

    # Filter warkops by input facilities
    for facility in facilities_set:
        if input_features.get(facility, 1) == 1:
            warkop_df_copy = warkop_df_copy[warkop_df_copy[facility] == 1]

    # Sort warkops by prediction score and then by rating
    sorted_warkops = warkop_df_copy.sort_values(by=['prediction', 'rating'], ascending=[False, False]).head(top_k)

    # Select the columns to return
    output_columns = ['name', 'rating', 'address','region'] + list(facilities_set)
    return sorted_warkops[output_columns]

In [ ]:
# Example input features for recommendation
input_features = {
    "wifi": 1,
    "24hours": 1,
    "ac": 1,
    "toilet": 1,
    "parking": 1
}

#Region filter
region = 'Barat'
recommended_warkops = recommend_warkop(input_features)
filtered_warkops = recommended_warkops.loc[warkop_df['region'] == region].copy()

# Get recommendations
# recommended_warkops
filtered_warkops

,name,rating,address,region,24hours,parking,wifi,ac,toilet
64,Warkop Ujung Aspal,5.0,"Jl. Surya Permata III No.79 12, RT.14/RW.1",Barat,1,1,1,1,1
69,Warkop Rasa Rindu,5.0,"Jl. Kali Sekretaris No.20, RT.8/RW.5",Barat,1,1,1,1,1
80,Warkop bang udin,5.0,"Jl. Mangga 19 No.2, RW.3",Barat,1,1,1,1,1
81,Warkop BASS,5.0,"Jl. Duri Mas 1 Ujung No.rt8, RT.5/RW.10",Barat,1,1,1,1,1
82,WARKOP MPOK NUNG,5.0,"Belakang Studio 5 Indosiar, Jl. Kali Sekretari...",Barat,1,1,1,1,1
62,Warkop Ropang Rai Raka,4.9,"Jl. Indraloka Raya No.28, RT.5/RW.6",Barat,1,1,1,1,1
51,Warkop Edo/edi,4.5,"Jl. Taman Ratu Raya Blk. BB1 No.19 10, RT.3/RW.13",Barat,1,1,1,1,1


In [ ]:
# Muat model yang sudah dilatih
model.save('warkop_model')
model_saved = tf.keras.models.load_model('warkop_model')

# Buat converter TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model_saved)
tflite_model = converter.convert()

# Simpan model TFLite
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


In [ ]:
# Print TensorFlow version
print('TensorFlow version:', tf.__version__)

# Print TensorFlow Lite version
print('TensorFlow Lite version:', tf.__version__)

TensorFlow version: 2.15.0
TensorFlow Lite version: 2.15.0


In [ ]:
import tensorflow as tf

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="/content/model.tflite")
interpreter.allocate_tensors()

# Check input and output details (optional)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Print input and output details
print("Input details:", input_details)
print("Output details:", output_details)

Input details: [{'name': 'args_0_4', 'index': 0, 'shape': array([1], dtype=int32), 'shape_signature': array([-1], dtype=int32), 'dtype': <class 'numpy.int64'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'args_0_9', 'index': 1, 'shape': array([1], dtype=int32), 'shape_signature': array([-1], dtype=int32), 'dtype': <class 'numpy.int64'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'args_0_11', 'index': 2, 'shape': array([1], dtype=int32), 'shape_signature': array([-1], dtype=int32), 'dtype': <class 'numpy.bytes_'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}

In [ ]:
def preprocess_input(input_features, input_details):
    input_data = {}
    for detail in input_details:
        name = detail['name']
        dtype = detail['dtype']
        index = detail['index']

        if dtype == np.int64:
            input_data[name] = np.array([input_features.get(name, 0)], dtype=np.int64)
        elif dtype == np.float32:
            input_data[name] = np.array([input_features.get(name, 0)], dtype=np.float32)
        elif dtype == np.bytes_:
            input_data[name] = np.array([str(input_features.get(name, '')).encode('utf-8')], dtype=np.bytes_)
        else:
            raise ValueError(f"Unsupported dtype {dtype} for input {name}")
    return input_data

In [ ]:
def recommend_warkop_tflite(input_features, facilities_set, top_k=100):
    warkop_df_copy = warkop_df.copy()

    predictions = []

    for i in range(len(warkop_df_copy)):
        row = warkop_df_copy.iloc[i]

        input_data = {facility: row[facility] for facility in facilities_set}
        input_data.update(input_features)

        processed_input = preprocess_input(input_data, input_details)

        for detail in input_details:
            interpreter.set_tensor(detail['index'], processed_input[detail['name']])

        interpreter.invoke()

        prediction = interpreter.get_tensor(output_details[0]['index'])

        predictions.append(prediction.flatten()[0])

    warkop_df_copy['prediction'] = predictions
    for facility in facilities_set:
        if input_features.get(facility, 1) == 1:
            warkop_df_copy = warkop_df_copy[warkop_df_copy[facility] == 1]

    sorted_warkops = warkop_df_copy.sort_values(by=['prediction', 'rating'], ascending=[False, False]).head(top_k)

    output_columns = ['name', 'rating', 'address', 'region'] + list(facilities_set)
    return sorted_warkops[output_columns]

In [ ]:
# Example input features for recommendation
input_features = {
    "24hours": 1,
    "ac": 1,
    "wifi": 1,
    "parking": 1,
    "toilet": 1
}

# Region filter
region = 'Timur'
recommended_warkops = recommend_warkop_tflite(input_features, facilities_set)
filtered_warkops = recommended_warkops[recommended_warkops['region'] == region]

filtered_warkops

,name,rating,address,region,24hours,parking,wifi,ac,toilet
135,Warkop Nyai Sinden,5.0,"Jl. H. Hanapi No.14, RT.14/RW.2",Timur,1,1,1,1,1
127,Warkop Maleber Jaya,4.5,Jalan Cipinang Lontar No. 1 Rt.06/Rw.06 Cipina...,Timur,1,1,1,1,1
96,Warkop SILMA,4.4,"No., Jl. Dermaga Raya No.322, RT.7/RW.9",Timur,1,1,1,1,1
116,Warkop Teduh,4.4,Jl. Cipinang Muara IV GG. Kutilang RT.002/011 ...,Timur,1,1,1,1,1
